# Extended USArray Preprocessing
This notebook is designed as a demonstration for the Earthscope 2025 MsPASS course section on HPC computing.   The notebook is designed to run as a batch process.   It assumes:
-  The job is being run from a working directory with required files at the or below that work directory in the file system
-  A "master" database with name "usarray48" has been created previously in this that directory. That db is assumed to contain a source, channel, and site collection
-  The working (current) directory contains a directory called *./wf/2012* containing miniseed files with names of the form *EventN.mseed" where N is a numeber (e.g. "Event4.mseed")

# Stage 1:  Build wf_miniseed 
This notebook is the first step for processing the extended usarray data set in year segments. It runs the indexing program to build a set of index documents in the wf_miniseed collection.  It then runs bulk_normalize to create the channel_id and site_id cross references in the wf_miniseed documents.   That is essential for the second stage of the processing.

This is a separate notebook because prototypes demonstrated:
1.  There are too many potential issues with miniseed data that can cause problems that is is useful to checkpoint the job at the end of the notebook to verify things are ok. That is particularly  true of normalizations and potential miniseed data problems.
2.  This notebook is note efficient to runw with a larger number of workers like the subsequent notebooks.   Running in with only 8 workers or so with a small memory requirement can be it through faster than waiting for a larger job requiring more resources, which is the case for the notebooks run after this one. 

In [1]:
# change for different calendar year
year = 2012
dbname = "usarray{}".format(year)
tsdir="./wf_TimeSeries/{}".format(year)
dbmaster="usarray48"

In [2]:
import mspasspy.client as msc
mspass_client = msc.Client()
# waveform data indexed to here
db = mspass_client.get_database(dbname)
# master database with source and receiver metadata
dbmd = mspass_client.get_database(dbmaster)


In [3]:
# comment out for normal use
import pymongo
mongoclient=pymongo.MongoClient()
mongoclient.drop_database(dbname)

In [4]:
# This builds a file list to drive index processing
import os
import fnmatch
import dask.bag as dbg
import time
topdirectory="./wf"
# assume year was defined at the top and data have the structure of wf/year/*.mseed
dir="{}/{}".format(topdirectory,year)
filelist=fnmatch.filter(os.listdir(dir),'*.mseed')
tstart=time.time()
data = dbg.from_sequence(filelist)
data = data.map(db.index_mseed_file,dir)
data=data.compute()
tend=time.time()
print("Elapsed time to run index_mseed_file=",tend-tstart)

Elapsed time to run index_mseed_file= 17.964977741241455


Normalization of the mseed records is complicated by the fact we are using two databases here.  The current normalize_mseed will not work because it assumes one db holds wf_miniseed and the channel-site collections.  For that reason I use bulk_normalize which uses a preloaded dataframe as input.  That, of course, is why the first part of this block loads the MiniseedMatcher.

In [5]:
from mspasspy.db.normalize import MiniseedMatcher,OriginTimeMatcher,bulk_normalize,normalize
# prepend_collection_name defaults to True but best to be clear that is essential here
chan_matcher = MiniseedMatcher(dbmd,
                               collection="channel",
                               prepend_collection_name=True,
                              )
site_matcher = MiniseedMatcher(dbmd,
                               collection="site",
                               attributes_to_load=["_id","lat","lon","elev","starttime","endtime"],
                               prepend_collection_name=True,
                              )
source_matcher = OriginTimeMatcher(dbmd,
                                   t0offset=0.0,
                                   tolerance=100.0,
                                   attributes_to_load=['_id','time'])
bulk_normalize(db,wf_col="wf_miniseed",matcher_list=[chan_matcher,site_matcher,source_matcher])

[178824, 177903, 177903, 178744]

In [6]:
# clear up these large objects 
del chan_matcher
del site_matcher
del source_matcher

Next we need to define some processing functions for specialized tasks in this workflow.   

In [7]:
from mspasspy.algorithms.window import WindowData
from mspasspy.algorithms.signals import detrend
from mspasspy.ccore.algorithms.basic import TimeWindow
from mspasspy.ccore.utility import ErrorSeverity
from mspasspy.algorithms.resample import (ScipyResampler,
                                          ScipyDecimator,
                                          resample,
                                         )
from mspasspy.db.normalize import ObjectIdMatcher
from mspasspy.algorithms.calib import ApplyCalibEngine
from mspasspy.util.seismic import number_live
from obspy import UTCDateTime
from obspy.geodetics import gps2dist_azimuth,kilometers2degrees
from obspy.taup import TauPyModel
from dask.distributed import as_completed
import time
import math



def set_PStime(d,Ptimekey="Ptime",Stimekey="Stime",model=None):
    """
    Function to calculate P and S wave arrival time and set times 
    as the header (Metadata) fields defined by Ptimekey and Stimekey.
    Tries to handle some complexities of the travel time calculator 
    returns when one or both P and S aren't calculatable.  That is 
    the norm in or at the edge of the core shadow.  
    
    :param d:  input TimeSeries datum.  Assumes datum's Metadata 
      contains stock source and channel attributes.  
    :param Ptimekey:  key used to define the header attribute that 
      will contain the computed P time.  Default "Ptime".
    :param model:  instance of obspy TauPyModel travel time engine. 
      Default is None.   That mode is slow as an new engine will be
      constructed on each call to the function.  Normal use should 
      pass an instance for greater efficiency.  
    """
    if d.live:
        if model is None:
            model = TauPyModel(model="iasp91") 
        # extract required source attributes
        srclat=d["source_lat"]
        srclon=d["source_lon"]
        srcz=d["source_depth"]
        srct=d["source_time"] 
        # extract required channel attributes
        stalat=d["channel_lat"]
        stalon=d["channel_lon"]
        staelev=d["channel_elev"]
        # set up and run travel time calculator
        georesult=gps2dist_azimuth(srclat,srclon,stalat,stalon)
        # obspy's function we just called returns distance in m in element 0 of a tuple
        # their travel time calculator it is degrees so we need this conversion
        dist=kilometers2degrees(georesult[0]/1000.0)
        arrivals=model.get_travel_times(source_depth_in_km=srcz,
                                            distance_in_degree=dist,
                                            phase_list=['P','S'])
        # always post this for as it is not cheap to compute
        # WARNING:  don't use common abbrevation delta - collides with data dt
        d['epicentral_distance']=dist
        # these are CSS3.0 shorthands s - station e - event
        esaz = georesult[1]
        seaz = georesult[2]
        # css3.0 names esax = event to source azimuth; seaz = source to event azimuth
        d['esaz']=esaz
        d['seaz']=seaz
        # get_travel_times returns an empty list if a P time cannot be 
        # calculated.  We trap that condition and kill the output 
        # with an error message
        if len(arrivals)==2:
            Ptime=srct+arrivals[0].time
            rayp = arrivals[0].ray_param
            Stime=srct+arrivals[1].time
            rayp_S = arrivals[1].ray_param
            d.put(Ptimekey,Ptime)
            d.put(Stimekey,Stime)
            # These keys are not passed as arguments but could be - a choice
            # Ray parameter is needed for free surface transformation operator
            # note tau p calculator in obspy returns p=R sin(theta)/V_0
            d.put("rayp_P",rayp)
            d.put("rayp_S",rayp_S)
        elif len(arrivals)==1:
            if arrivals[0].name == 'P':
                Ptime=srct+arrivals[0].time
                rayp = arrivals[0].ray_param
                d.put(Ptimekey,Ptime)
                d.put("rayp_P",rayp)
            else:
                # Not sure we can assume name is S
                if arrivals[0].name == 'S':
                    Stime=srct+arrivals[0].time
                    rayp_S = arrivals[0].ray_param
                    d.put(Stimekey,Stime)
                    d.put("rayp_S",rayp_S)
                else:
                    message = "Unexpected single phase name returned by taup calculator\n"
                    message += "Expected phase name S but got " + arrivals[0].name
                    d.elog.log_error("set_PStime",
                                     message,
                                     ErrorSeverity.Invalid)
                    d.kill()
        else:
            # in this context this only happens if no P or S could be calculated
            # That shouldn't ever happen but we need this safety in he event it does
            message = "Travel time calculator failed completely\n"
            message += "Could not calculator P or S phase time"
            d.elog.log_error("set_PStime",
                             message,
                             ErrorSeverity.Invalid)
            d.kill()
    return d




def set_file_path(e,dir="wf_TimeSeries",ext=None):
    """
    This function is used to set dir and dfile for this workflow for each 
    ensemble.  Note these are set in the ensemble's metadata container 
    not the members.   We don't set them for members as Database.save_data 
    only uses kwarg dir and dfile with the metadata as a fallback.
    """
    e['dir']=dir
    dfile = "dfile_undefined"
    for d in e.member:
        if d.live and 'dfile' in d:
            dfile=d['dfile']
            sret = dfile.split(".")
            # split returns a list.  If there is no extension only one is returned 
            # in 0 but that is all we need here
            dfile=sret[0]
            break
    if ext is not None:
        dfile = dfile + "." + ext
    return e

def dbmdquery(year,padsize=86400.0):
    """
    Constructs a MongoDB query dictionary to use as a query argument for normalization matcher classes.
    (All standard BasicMatcher children have a query argument in the constructor for this purpose.)
    The query is a range spanning specified calendar year.   The interval is extended by padsize 
    seconds.   (default is one day = 86400.0)

    Note this query is appropriate for channel not site.
    """
    # uses obspy's UTCDateTime class to create time range in epoch time using the 
    # calendar strings for convenience
    tstr = "{}-01-01T00:00:00.0".format(year)
    st = UTCDateTime(tstr)
    starttime = st.timestamp - padsize
    tstr = "{}-01-01T00:00:00.0".format(year+1)
    et = UTCDateTime(tstr)
    endtime = et.timestamp + padsize
    # not sure the and is required but better safe than sorry
    query = { "$and" :
             [
                 {"starttime" : {"$lte" : endtime}},
                 {"endtime" : {"$gte" : starttime}}
             ]
    }
    return query

def atomic_ts_processor(d,decimator,resampler,ttmodel,win,calib_engine):
    """
    This function puts all the processing functions for input TimeSeres (d).  It is called in this 
    workflow in a map operator applied to ensemble members.  
    """
    d = detrend(d,type="constant")
    d = resample(d,decimator,resampler)
    d = set_PStime(d,model=ttmodel)
    if d.live and "Ptime" in d:
        ptime = d["Ptime"]
        d.ator(ptime)
        d = WindowData(d,win.start,win.end)
        d.rtoa()
    d = calib_engine.apply_calib(d)
    return d   

This is a major run section.  It constructs a parallel container of `TimeSeriesEnsemble` objects from miniseed data in the `read_distributed_data` line.  It then runs a sequence of algorithms on each ensemble aiming mainly to window the data down to shorter time segments focused on the P wave arrival time.  It then saves the preprocessed result as files in wf_TimeSeries.   This is the most time consuming section of this workflow.

In [8]:
from mspasspy.ccore.seismic import TimeSeriesEnsemble
from mspasspy.io.distributed import read_distributed_data
ttmodel = TauPyModel(model="iasp91")
t0 = time.time()
# important to note these matchers are loaded from dbmd
# bulk_normalize above will set the matching ids
timequery = dbmdquery(year)
chan_matcher = ObjectIdMatcher(dbmd,
                               query=timequery,
                               collection="channel",
                               attributes_to_load=["_id","lat","lon","elev","hang","vang"],
                              )
tstr1 = "{}-01-01T00:00:00.0".format(year)
tstr2 = "{}-01-01T00:00:00.0".format(year+1)
st1=UTCDateTime(tstr1)
st2=UTCDateTime(tstr2)
srcquery = {"time" : {"$gte" : st1.timestamp, "$lt" : st2.timestamp}}
source_matcher = ObjectIdMatcher(dbmd,
                                query=srcquery,
                                collection="source",
                                attributes_to_load=["_id","lat","lon","depth","time"],
                                load_if_defined=["magnitude"],
                                )
target_sample_rate=20.0
resampler=ScipyResampler(target_sample_rate)
decimator=ScipyDecimator(target_sample_rate)
calib_engine = ApplyCalibEngine(dbmd)
tswin = TimeWindow(-200.0,500.0)
srcidlist_ms=db.wf_miniseed.distinct('source_id')

srcidlist_finished = db.wf_TimeSeries.distinct('source_id')
if len(srcidlist_finished)>0:
    srcidlist=[]
    for sid in srcidlist_ms:
        if sid not in srcidlist_finished:
            srcidlist.append(sid)
else:
    srcidlist = srcidlist_ms


print("Number of distinct sources in wf_miniseed= ",len(srcidlist_ms))
print("Number to process this run=",len(srcidlist))

for sid in srcidlist:
    print("working on  ",sid)
    query = {"source_id" : sid}
    mydata = read_distributed_data(db,
                                   query=query,
                                   collection="wf_miniseed",
                                   normalize=[chan_matcher,source_matcher],
                                   npartitions=60,
                                  )
    mydata=mydata.map(atomic_ts_processor,
                            decimator,
                            resampler,
                            ttmodel,
                            tswin,
                            calib_engine)
    dlist = mydata.compute()
    # package into ensemble for a faster write 
    ens=TimeSeriesEnsemble(len(dlist))
    for d in dlist:
        ens.member.append(d)
    # needed because we constructed this manual from a list
    ens.set_live()
    ens = set_file_path(ens,dir=tsdir)
    ens = db.save_data(ens,
                       return_data=True,
                       collection="wf_TimeSeries",
                       storage_mode="file",
                       dir=tsdir,
                       data_tag="preprocessed_map",
                       )
    del ens
    
    
t = time.time()
print("Time to process from raw data to TimeSeries objects in db = ",t-t0," seconds")

Number of distinct sources in wf_miniseed=  56
Number to process this run= 56
working on   67e675c7878ef7c6d337c8bd


/opt/conda/lib/python3.10/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 12.88 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


working on   67e675c7878ef7c6d337c8cb


/opt/conda/lib/python3.10/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 12.87 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


working on   67e675c7878ef7c6d337c90b


...

working on   67e675c9878ef7c6d337d7f9


/opt/conda/lib/python3.10/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 12.79 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


working on   67e675c9878ef7c6d337d813
working on   67e675c9878ef7c6d337d858
working on   67e675c9878ef7c6d337d859
working on   67e675c9878ef7c6d337d85f
working on   67e675c9878ef7c6d337d860
working on   67e675c9878ef7c6d337d862
working on   67e675c9878ef7c6d337d88d
working on   67e675c9878ef7c6d337d8c2
working on   67e675c9878ef7c6d337dab4
working on   67e675c9878ef7c6d337dc3c


/opt/conda/lib/python3.10/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 12.76 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


working on   67e675ca878ef7c6d337dd52
working on   67e675ca878ef7c6d337dd56
working on   67e675ca878ef7c6d337de2b
working on   67e675ca878ef7c6d337df11
working on   67e675ca878ef7c6d337e19e
working on   67e675ca878ef7c6d337e230
working on   67e675ca878ef7c6d337e24e
Time to process from raw data to TimeSeries objects in db =  2061.5057950019836  seconds


## Create and Store Seismogram objects
This next section reads back the data we just created, converts all the data to `Seismogram` objects, runs the free-surface transformation operator, and finally runs an snr based quality control editor that will automatically kill data with no detectable signal.  This section takes a lot of compute time but only about 1/3 as long as the previous major step.

First define new functions needed for this processing:

In [9]:
from mspasspy.db.normalize import normalize
from mspasspy.algorithms.bundle import bundle_seed_data
from mspasspy.ccore.seismic import SlownessVector
from mspasspy.algorithms.basic import free_surface_transformation,rotate_to_standard
from mspasspy.algorithms.snr import broadband_snr_QC
from mspasspy.util.seismic import number_live
import dask.bag as dbg
import time
import math

def read_tsensembles(source_id,db,data_tag=None):
    """
    Reader for this script. Could be done with read_distributed_data and a query generator 
    to create a list of python dictionaries as input to read_distributed_data.   This does the 
    same thing more or less and is a bit clearer.
    
    """
    print("Debug:  entered read_tsensembles")
    query = {"source_id" : source_id}
    if data_tag:
        query["data_tag"] = data_tag
    n = db.wf_TimeSeries.count_documents(query)
    print("Found ",n," documents for nsemble with source_id=",source_id)
    cursor = db.wf_TimeSeries.find(query)
    ens = db.read_data(cursor,collection="wf_TimeSeries")
    print("Number of members in this ensemble=",len(ens.member))
    return ens

def normalize_ensembles(ens,matcher):
    """
    Workaround for bug in normalize function where decorators won't work.
    This function should be removed when that bug is resolved.
    """
    if ens.live:
        for i in range(len(ens.member)):
            ens.member[i] = normalize(ens.member[i],matcher)
    return ens

def apply_FST(d,rayp_key="rayp_P",seaz_key='seaz',vp0=6.0,vs0=3.5):
    """
    Apply free surface transformation operator of Kennett (1983) to an input `Seismogram` 
    object.   Assumes ray parameter and azimuth data are stored as Metadata in the 
    input datum.  If the ray parameter or azimuth key are not defined an error 
    message will be posted and the datum will be killed before returning.  
    :param d:  datum to process
    :type d:  Seismogram
    :param rayp_key:   key to use to extract ray parameter to use to compute the 
    free surface transformation operator.  Note function assumes the ray parameter is
    spherical coordinate form:  R sin(theta)/V.   Default is "rayp_P".
    :param seaz_key:   key to use to extract station to event azimuth. Default is "seaz".
    :param vp0:  surface P wave velocity (km/s) to use for free surface transformation 
    :param vs0:  surface S wave velocity (km/s) to use for free surface transformation.
    """
    if d.is_defined(rayp_key) and d.is_defined(seaz_key):
        rayp = d[rayp_key]
        seaz = d[seaz_key]
        # Some basic seismology here.  rayp is the spherical earth ray parameter
        # R sin(theta)/v.  Free surface transformation needs apparent velocity 
        # at Earth's surface which is sin(theta)/v when R=Re.   Hence the following
        # simple convertion to get apparent slowness at surface  sin(theta)/v
        Re=6378.1
        umag = rayp/Re   # magnitude of slowness vector
        # seaz is back azimuth - slowness vector points in direction of propagation
        # with is 180 degrees away from back azimuth
        az = seaz + 180.0
        # component slowness vector components in local coordinates
        ux = umag * math.sin(az)
        uy = umag * math.cos(az)
        # FST implementation requires this special class called a Slowness Vector
        u = SlownessVector(ux,uy)
        d = free_surface_transformation(d,uvec=u,vp0=vp0,vs0=vs0)
    else:
        d.kill()
        message = "one of required attributes rayp_P or seaz were not defined for this datum"
        d.elog.log_error("apply_FST",message,ErrorSeverity.Invalid)
        
    return d



def process2seis(ens,
            swin,
            nwin,
            signal_engine,
            noise_engine,
                ):
    """
    This is the master processing function for this step.   It assumes an input of ens 
    that is a common source gather with required Metadata set in the earlier run with the stage 2 
    (Preprocess2ts) notebook.  In this workflow ens is read in parallel with this functiond oing 
    the processing.  The output is a SeismogramEnsemble run through broadband_snr_QC to set 
    snr metrics.  It also normally kills a large fraction of the data.  

    The approach using a loop over the ensemble was designed to reduce the memory footprint as 
    each member is processed and replaces its parent in the ensemble that is returned.  
    """
    print("Processing TimeSeriesEnsemble with ",len(ens.member)," members")
    ens3c = bundle_seed_data(ens)
    del ens
    N = len(ens3c.member)
    for i in range(N):
        ens3c.member[i] = apply_FST(ens3c.member[i])
        ens3c.member[i] = broadband_snr_QC(ens3c.member[i],
                            component=2,
                            signal_window=swin,
                            noise_window=nwin,
                            use_measured_arrival_time=True,
                            measured_arrival_time_key="Ptime",
                            noise_spectrum_engine=noise_engine,
                            signal_spectrum_engine=signal_engine,
                                )
    return ens3c
    

This block run the functions defined above in combination with several MsPASS auxiliary objects.  It constructs a bag of `TimeSeriesEnsemble` objects  and passes it through the processing functions described above to produce there result.  That is, and edited set of `Seismogram` objects stored as files in "wf_Seismogram" with Metadata stored in the database in the collection "wf_Seismogram" (Note the confusion that the same name is used for the collection and a directory for holding the sample data in files).  Emphasize the "edited" as the output of this sequence is commonly 1/10 the size of the input. It leaves a lot of bodies in the "cemetery" collection.

In [10]:
from mspasspy.algorithms.MTPowerSpectrumEngine import MTPowerSpectrumEngine
wfdir = "./wf_Seismogram/{}".format(year)
t0 = time.time()
#chanqmspasspy.algorithms.MTPowerSpectrumEngine.MTPowerSpectrumEngineuery=dbmdquery(year)
chan_matcher = ObjectIdMatcher(dbmd,
                               collection="channel",
                               attributes_to_load=["_id","lat","lon","elev","hang","vang"],
                              )

# for usage on IU system each node has 2*64=128 cores so using 128 workers for the run
# makes this twice that
npartitions=100
swin = TimeWindow(-5.0,100.0)
nwin = TimeWindow(-195.0,-5.0)
# spectrum estimation engines for broadband_snr_QC
dt = 0.05
nsamp_noise = int((nwin.end-nwin.start)/dt) + 1
nsamp_sig = int((swin.end-swin.start)/dt) + 1
signal_engine=MTPowerSpectrumEngine(nsamp_sig,5.0,8,2*nsamp_sig,dt)
noise_engine=MTPowerSpectrumEngine(nsamp_noise,5.0,10,2*nsamp_noise,dt)
srcidlist_ms=db.wf_TimeSeries.distinct('source_id')

srcidlist_finished = db.wf_Seismogram.distinct('source_id')
if len(srcidlist_finished)>0:
    srcidlist=[]
    for sid in srcidlist_ms:
        if sid not in srcidlist_finished:
            srcidlist.append(sid)
else:
    srcidlist = srcidlist_ms


print("Number of distinct sources in wf_TimeSeries= ",len(srcidlist_ms))
print("Number to process this run=",len(srcidlist))
# reduce size for testing 
#sidtmp=[]
#for i in range(4):
#    sidtmp.append(srcidlist[i])
#srcidlist=sidtmp
#print("Debug process list:  ",srcidlist)
mydata = dbg.from_sequence(srcidlist)
mydata = mydata.map(read_tsensembles,db)
# note default behavior for normalize is to normalize all members
#mydata = mydata.map(normalize,chan_matcher,handles_ensembles=False)
#workaround for above until bug is fixed in normalize
mydata = mydata.map(normalize_ensembles,chan_matcher)
mydata = mydata.map(process2seis,
                      swin,
                      nwin,
                      signal_engine,
                      noise_engine,
                               )
mydata = mydata.map(set_file_path,dir=wfdir,ext="3C")
mydata = mydata.map(db.save_data,
                    collection="wf_Seismogram",
                    storage_mode="file",
                    dir=wfdir,
                    data_tag="FST_and_QCed",
                   )
out=mydata.compute()
print(out)
t = time.time()
print("Run time = ",t-t0," seconds")

Number of distinct sources in wf_TimeSeries=  52
Number to process this run= 52


/opt/conda/lib/python3.10/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 27.30 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


[{'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': False}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': False}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': False}, {'is_live': True}, {'is_live': False}, {'is_live': False}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}, {'is_live': True}]
Run ti

# Receiver Function Estimation
The last step in this workflow is to compute receiver function estimates from all the Seismogram objects that survived winnowing with the broadband_snr_QC function.  The estimation method is a novel method called "Colored Noise Regularized Deconvolution" (CNRDecon).   The method can be viewed as form of multitaper deconvolution where the bandwidth of the data is determined dynamically for each Seismogram object.   To validate the data this code saves the "actual output" of each deconvolution operator in wf_TimeSeries.

In [11]:
from mspasspy.ccore.utility import AntelopePf
from mspasspy.ccore.seismic import TimeSeries
from mspasspy.ccore.algorithms.deconvolution import CNRDeconEngine
from mspasspy.ccore.algorithms.amplitudes import MADAmplitude
from mspasspy.algorithms.basic import ExtractComponent
from mspasspy.algorithms.CNRDecon import CNRRFDecon
from mspasspy.util.Undertaker import Undertaker
import numpy as np

This box defines a couple composite functions used in the processing sequenc below.

In [12]:
def remove_incident(d,ao,nw_end=-3.0,P_component=2):
    if d.dead():
        return d
    stime = max(d.t0,ao.t0)
    etime = min(d.endtime(),ao.endtime())
    x_ao = WindowData(ao,stime,etime)
    s_amp = np.zeros(3)
    n_amp = np.zeros(3)
    snr = np.zeros(3)
    for i in range(3):
        x = ExtractComponent(d,i)
        x_n = WindowData(x,d.t0,nw_end)
        n_amp[i] = MADAmplitude(x_n)
        x_w = WindowData(x,stime,etime)
        amp = np.dot(x_ao.data,x_w.data)
        s_amp[i] = amp
        if n_amp[i]>0.0:
            snr[i] = s_amp[i]/n_amp[i]
        else:
            if s_amp[i]>0.0:
                snr[i]=9999999.9
            else:
                s_amp[i]=1.0
        scaled_ao = TimeSeries(x_ao)
        # Note -amp used so when we add scaled it is subtracting incident
        scaled_ao *= -amp
        # operator += handles irregular start and end time
        # taper may be advised but in this context probably necessary
        x += scaled_ao
        d.data[i,:] = x.data
        
    # post snr metrics before exiting
    snrdoc=dict()
    # not sur mongodb will handle a np array corectly so convert this 
    # to a python list
    snrlist=list()
    namplist=list()
    for i in range(3):
        snrlist.append(snr[i])
        namplist.append(n_amp[i])
    snrdoc['component_snr']=snrlist
    snrdoc['component_noise']=namplist
    # vector sum of incident horizontal snr a more useful metric than 
    # components
    sig_H = 0.0
    noise_H = 0.0
    for i in range(3):
        if i != P_component:
            sig_H += s_amp[i]*s_amp[i]
            noise_H += n_amp[i]*n_amp[i]
    sig_H = np.sqrt(sig_H)
    noise_H = np.sqrt(noise_H)
    if noise_H < np.finfo(float).eps:
        if sig_H < np.finfo(float).eps:
            snr=1.0
        else:
            snr = sig_H/np.finfo(float).eps
    else:
        snr = sig_H/noise_H
    snrdoc['snr_H'] = snr
    d['snr_RF'] = snrdoc
    return d

def process(sid,db,dbmd,engine,signal_window,noise_window,data_tag=None)->int:
    """
    Process one SeismogramEnsemble for source id sid with CRNDecon algorithm. 
    Saves output in two forms.   All live outputs from the decon algorithmn 
    (CNRRFDecon can kill) are saved with data_tag="CNRRFDecon_data_raw".  
    The algorithm then calls the function "remove_incident" (see above) 
    to minimize direct P wave energy in all three components.  
    (Note with an RF that means the P component will be very small
    except at the fringes of the data window)  Finally, it computes the 
    actual output and ideal output wavelets and stores them in the 
    wf_TimeSeries with cross-references to the _id of the saved RF estimates.
    They are stored with data_tag of "CNRRFDecon_ao" and "CNRRFDecon_io" 
    respectively.

    :param sid:  source_id to process
    :param db:  waveform database managing Seismogram objects to be processed
    :param dbmd:  Metadata database - in this implementation not the same as db
    :param engine:  Instance of CNRRFDeconEngine.  This algorithm has a 
      fairly high initialization cost.  The engine avoids that initialziation
      cost by encapsulating all the things required to run the algorithm 
      in this object.
    :param signal_window:  TimeWindow object of section of data to treat 
      as signal
    :param noise_window:  TimeWindow object defining section of data to 
      treat as noise (used for computing regularization so must overlap 
      data range).
    :param data_tag:  data tag to subset INPUT not output.   When set 
      (default is None) the query by sid will append a data_tag 
      quality match for the value received.

    :return:  tuple that can just be printed.  Content is: 
      [str(sid), number of ensemble members, number marked live]
    """
    query = {'source_id' : sid}
    ntotal = db.wf_Seismogram.count_documents(query)
    query['Parrival.median_snr'] = {'$gt' : 2.0}
    query['Parrival.snr_filtered_envelope_peak'] = {'$gt' : 3.0}
    query['Parrival.bandwidth'] = {'$gt' : 8.0}
    if data_tag:
        query['data_tag'] = data_tag
    cursor=db.wf_Seismogram.find(query)
    ens = db.read_data(cursor,collection="wf_Seismogram")
    print('source_id=',sid,' Processing ',len(ens.member),' of ',ntotal)
    nlive = 0
    nmembers = len(ens.member)
    for d in ens.member:
        if d.live:
            Ptime=d['Ptime']
            d.ator(Ptime)
            decon_output = CNRRFDecon(d,
                        engine,
                        signal_window=signal_window,
                        noise_window=noise_window,
                        bandwidth_subdocument_key="Parrival",
                        return_wavelet=True,
                       )
            rf0 = decon_output[0]
            if rf0.dead():
                # it is a bit inefficient to instantiate an instance of 
                # an Undertaker for each error but CMRRFDecon exceptions 
                # are not common unless the engine is badly configured.
                stedronsky = Undertaker(dbmd)
                stedronsky.bury(rf0)
            else:
                ao = decon_output[1]
                io = decon_output[2]
                rf = remove_incident(rf0, ao)
                # temp for debug with spyder - no save
                #if rf:
                    #plotter=SeismicPlotter(normalize=True,style='wt')
                    #plotter.plot(rf)
                    #plt.show()
                    #continue
                dir="wf_RF"
                s=rf['dfile']
                dfile = "RF"+s
                rf0=dbmd.save_data(rf0,
                              collection='wf_Seismogram',
                              return_data=False,
                              storage_mode='file',
                              dir=dir,
                              dfile=dfile,
                              data_tag='CNRRFDecon_data_raw')
                rf=dbmd.save_data(rf,
                              collection='wf_Seismogram',
                              return_data=True,
                              storage_mode='file',
                              dir=dir,
                              dfile=dfile,
                              data_tag='CNRRFDecon_data')
            
                if rf.live:
                    wfid = rf['_id']
                    ao['parent_wfid'] = wfid
                    io['parent_wfid'] = wfid
                    dfile = "ao_" + s
                    dbmd.save_data(ao,collection='wf_TimeSeries',storage_mode='file',dir=dir,dfile=dfile,data_tag='CNRRFDecon_ao')
                    dfile = "io_" + s
                    dbmd.save_data(io,collection='wf_TimeSeries',
                             storage_mode='file',
                             dir=dir,dfile=dfile,
                             data_tag='CNRRFDecon_io')
                    nlive += 1
        else:
            print("WARNING:  ensemble member was marked dead - has to be an abortion")
            nlive = 0
        return [str(sid),nmembers,nlive]

Now run the decon algorithm.  Note the approach used here is different.  It demonstrates the use of concurrent "Futures" in dask.   Note the __[dask Futures API](https://docs.dask.org/en/latest/futures.html)__ is similar to the __[threaded version](https://docs.python.org/3/library/asyncio-future.html)__ that is now standard python. 

In [13]:
pf=AntelopePf('CNRDeconEngine.pf')
engine = CNRDeconEngine(pf)
signal_window=TimeWindow(-10.0,100.0)
noise_window=TimeWindow(-200.0,-5.0)

sidlist=db.wf_Seismogram.distinct('source_id')
t0=time.time()
# Serial version - acceptable for tutorial as run time is short
nprocessed=0
for sid in sidlist:
    print("Working on source_id=",sid)
    rfens = process(sid,db,dbmd,engine,signal_window,noise_window)
    nprocessed+=1

# Something sent to process does not serialize so either of the parallelizations
# below fail complaining of a serialization error.

#import dask.bag as dbg
#mydata=dbg.from_sequence(sidlist)
#mydata = mydata.map(process,db,dbmd,engine,signal_window,noise_window)
#mydata.compute()

# this demonstrates parallelization of the loop above using Futures approach
#dask_client = mspass_client.get_scheduler()
#sidlist=db.wf_Seismogram.distinct("source_id")
# this will contain a collection of Futures objects that are data ruuning or 
# queued for processing with the "process" function.  
#futures_list=[]
#for sid in sidlist:
#    f = dask_client.submit(process,sid,db,dbmd,engine,signal_window,noise_window)
#    futures_list.append(f)
# wait for output and print the return values
# this construct is not necessary for this algorithm but demonstrates that one 
# can run a serial section assynchronously as futures return 
# i.e. output will not be in the order submitted.
#print("source_id n_in n_saved")
#for future in as_completed(futures_list):
#    out = future.result()
#    print(out)
    
t=time.time()
print("Elapsed time to compute RF estimates=",t-t0)

Working on source_id= 67e675c7878ef7c6d337c8bd
source_id= 67e675c7878ef7c6d337c8bd  Processing  932  of  1036
Working on source_id= 67e675c7878ef7c6d337c8cb
source_id= 67e675c7878ef7c6d337c8cb  Processing  321  of  357
Working on source_id= 67e675c7878ef7c6d337c90f
source_id= 67e675c7878ef7c6d337c90f  Processing  22  of  51
Working on source_id= 67e675c7878ef7c6d337c990
source_id= 67e675c7878ef7c6d337c990  Processing  363  of  533
Working on source_id= 67e675c7878ef7c6d337c992
source_id= 67e675c7878ef7c6d337c992  Processing  1208  of  1216
Working on source_id= 67e675c7878ef7c6d337caca
source_id= 67e675c7878ef7c6d337caca  Processing  1102  of  1153
Working on source_id= 67e675c7878ef7c6d337cafa
source_id= 67e675c7878ef7c6d337cafa  Processing  1072  of  1143
Working on source_id= 67e675c7878ef7c6d337cb2e
source_id= 67e675c7878ef7c6d337cb2e  Processing  681  of  813
Working on source_id= 67e675c7878ef7c6d337cb50
source_id= 67e675c7878ef7c6d337cb50  Processing  602  of  650
Working on sou